In [ ]:
import numpy as np

from LogicalQ.Logical import LogicalCircuit
from LogicalQ.Library.QECCs import steane_code
from LogicalQ.Library.HardwareModels import hardware_models_Quantinuum
from LogicalQ.NoiseModel import construct_noise_model_from_hardware_model
from LogicalQ.Optimizers import compute_effective_threshold, compute_gadget_costs, compute_constraint_model
from LogicalQ.Benchmarks import mirror_benchmarking
from LogicalQ.Experiments import execute_circuits, circuit_sclaing_experiment, qec_cycle_efficiency_experiment
from LogicalQ.Analysis import qec_cycle_efficiency_scatter

## Compute the effective threshold

In [ ]:
_, _, effective_threshold = compute_effective_threshold(
    hardware_model=hardware_models_Quantinuum["H2-1"],
    **steane_code,
    min_theta=0, max_theta=np.pi/2, n_theta=8,
    min_phi=0, max_phi=np.pi*2, n_phi=4,
    max_n_qec_cycles=8
)

## Compute gadget costs

In [ ]:
gadgets_library, gadget_infidelities, gadget_costs = compute_gadget_costs(
    gadgets_library=None,
    backend="aer_simulator", hardware_model=hardware_models_Quantinuum["H2-1"]
)

## Construct constraint model

In [ ]:
compute_constraint_model(
    hardware_models_Quantinuum["H2-1"],
    **steane_code,
    optimizer=True,
    effective_threshold=effective_threshold,
    gadget_costs=gadget_costs,
    constraint_model=None,
)

## Test constraint model

In [ ]:
def logical_circuit_factory(n_qubits, circuit_length):
    qc = mirror_benchmarking(n_qubits=n_qubits, circuit_length=circuit_length, seed=1111)
    return LogicalCircuit.from_physical_circuit(qc, **steane_code)

In [ ]:
# @TODO - replace with qec_cycle_circuit_scaling_experiment once implemented
data = circuit_scaling_experiment(
    logical_circuit_factory, noise_model_factory,
    min_n_qubits=1, max_n_qubits=1,
    min_circuit_length=1, max_circuit_length=32,
    shots=1E6,
    with_mp=True
)

In [ ]:
circuit_scaling_bar3d(data)